In [18]:
import os
import git
from pathlib import Path
from typing import List
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import plotly.graph_objects as go
from IPython.display import clear_output
import scipy
import pylustrator
import math

ROOT_DIR =  Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
SAVE_FIGS = False

os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(os.path.join(ROOT_DIR, "results", "combined_results"))
np.random.seed(0)
plots_path = os.path.join(ROOT_DIR, "publication", "paper", "draft_plots")

def variance_prior(r, eta, scale=1):
    beta = (eta+1.5)/r
    var_prior = scale * scipy.special.gamma(beta + 1/r)/scipy.special.gamma(beta)
    return var_prior

def kurtosis_prior(r, eta, fisher=True):
    beta = (eta+1.5)/r
    kurtosis = 3*scipy.special.gamma(beta + 2/r)*scipy.special.gamma(beta)/scipy.special.gamma(beta+1/r)**2 
    if fisher:
        return kurtosis - 3
    else:
        return kurtosis 

fixed_palette = {
    'gray': 'xkcd:gray',
    'green': 'xkcd:shamrock green',
    'red': 'xkcd:light red',
    'blue': 'xkcd:blue',
}

def find_master_dfs(root_dir: str) -> List[str]:
    root_path = Path(root_dir)
    if not root_path.exists():
        raise FileNotFoundError(f"Directory not found: {root_dir}")

    master_df_paths = []
    for current_dir, _, files in os.walk(root_path):
        if 'master_df.csv' in files:
            master_df_path = Path(os.path.join(current_dir, 'master_df.csv'))
            master_df_paths.append(str(master_df_path.absolute()))
    return master_df_paths

def add_hull(master_df, rEtaKsstats_dict, GROUP='group', debug=False):

    master_df_copy = master_df.copy()
    master_df_copy = master_df.set_index(GROUP)
    groups = master_df_copy.index
    master_df_copy["hull"] = ""

    for group in groups:
        if master_df_copy.loc[group, "total_samples"] < 10:
            master_df_copy.loc[group, "hull"] = np.nan
           
        else:
            drop_keys =list(rEtaKsstats_dict[group].keys())[-3:]
            if debug:
                print(drop_keys)
            pre_optimization = pd.DataFrame(rEtaKsstats_dict[group]).drop(drop_keys, axis = 1 )
            optimization = pd.DataFrame(rEtaKsstats_dict[group])[drop_keys]
            optimization = optimization.rename(columns = {"r_optimize": "r", "eta_optimize": "eta", drop_keys[-1]: "ksstat"})
            optimization = optimization.dropna()
            full_df = pre_optimization.merge(optimization, on=["r", "eta"], how="outer")
            full_df = full_df.set_index(["r", "eta"])
            full_df["ksstat"] = full_df.min(axis=1)
            full_df = full_df.reset_index()
            full_df = full_df[["r", "eta", "ksstat"]]
            full_df["1/beta"] = full_df["r"]/(full_df["eta"] + 1.5)
            MULT = 1.2
            cutoff = max(min(full_df["ksstat"]) * MULT, master_df_copy.loc[group, "kstest_stat_cutoff_0.05"], 0.01)
            filtered_df = full_df[full_df["ksstat"] < cutoff]
            points = np.column_stack((filtered_df["r"], filtered_df["1/beta"])) + stats.norm.rvs(size=(len(filtered_df), 2)) * 0.001  # Adding small noise for convex hull computation
            hull = ConvexHull(points)
            master_df_copy.loc[group, "hull"] = hull

    return master_df_copy.reset_index()

relevant_cols = ['group', 'obs_var', 'var_lower', 'var_upper', 'obs_kurt', 'kurt_lower',
       'kurt_upper', 'total_samples', 'initial_r', 'initial_eta', # 'initial_scale',
       'kstest_stat_initial', 'kstest_stat_cutoff_0.05',
       'best_r', 'best_eta', # 'best_scale', 
       'kstest_stat_best', # 'n_pval_0.05',
        'param_gaussian',
       'kstest_stat_gaussian', 'kstest_pval_gaussian', 'param_laplace',
       'kstest_stat_laplace', 'kstest_pval_laplace', 'param_t',
       'kstest_stat_t', 'kstest_pval_t', 'kstest_pval_gengamma', 
       'dataset', 'subset', 'transform', 'orientation', 'channel', 'github_plot', 'dataset_type', 'hull']

all_paths = find_master_dfs(os.path.join(ROOT_DIR, "results", "case-studies"))
all_master_dfs = []
github_plots_path = "https://github.com/yashdave003/hierarchical-bayesian-model-validation/blob/main/results/case-studies/"

for path in all_paths:
    if 'scaleTesting' in path:
        continue
    master_df = pd.read_csv(path)
    master_df = master_df.rename(columns={master_df.columns[0]: 'group'})
    parts = Path(path).parts[-7:]
    if parts[0] == 'case-studies':
        parts = parts[1:]
    if "MRI" in path:
        dataset, slice, transform, orientation, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = slice
        master_df['channel'] = np.nan
        master_df['orientation'] = orientation
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, slice, transform, orientation, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    
    elif len(parts) > 6:
        dataset, subset, transform, orientation, channel, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = subset
        master_df['channel'] = channel
        master_df['orientation'] = orientation
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, subset, transform, orientation, channel, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    else:
        dataset, size, transform, channel, _, _ = parts
        master_df['dataset'] = dataset
        master_df['transform'] = transform
        master_df['subset'] = size
        master_df['channel'] = channel
        master_df['orientation'] = np.nan
        master_df['github_plot'] = [github_plots_path+'/'.join([dataset, size, transform, channel, 'plots', f'compare_cdf_pdf_layer_{group}.jpg']) for group in master_df['group']]
    
    if dataset in ['pastis', 'agriVision', 'spaceNet']:
        master_df['dataset_type'] = 'remote sensing'
    elif dataset in ['syntheticMRI2D', 'syntheticMRI3D']:
        master_df['dataset_type'] = 'medical'
    elif dataset in ['coco', 'segmentAnything']:
        master_df['dataset_type'] = 'natural'

    GROUP = 'layer' if transform.split("-")[0] == 'wavelet' else ('band' if transform.split("-")[0] == 'fourier' else 'error')
    rEtaKsstatsDict = pd.read_pickle(path[:-18] + "cache/rEtaKsstats_dict.pickle")
    master_df = add_hull(master_df, rEtaKsstatsDict)


    all_master_dfs.append(master_df[relevant_cols])
    
main_df = pd.concat(all_master_dfs)
# main_df['prior_var'] = main_df.apply(lambda row : variance_prior(row.loc['best_r'], row.loc['best_eta']), axis = 1)
# main_df['prior_kurt'] = main_df.apply(lambda row : kurtosis_prior(row.loc['best_r'], row.loc['best_eta']), axis = 1)
# main_df['gaussian_kurt'] = np.zeros(main_df.shape[0])
# main_df['laplace_kurt'] = 3*np.ones(main_df.shape[0])
main_df['best_beta'] = (main_df['best_eta'] + 1.5)/main_df['best_r'] 
main_df['best_1/beta'] = 1/main_df['best_beta']
main_df['total_samples'] = main_df['total_samples']//10 # TODO: look into why total_samples*10
main_df['kstest_stat_cutoff_0.10'] = stats.kstwo(n=main_df['total_samples']).isf(0.1)
kstest_stat_cutoff_2 = 0.2
kstest_stat_cutoff_2_name = 'kstest_stat_cutoff_0.20'
main_df[kstest_stat_cutoff_2_name] = stats.kstwo(n=main_df['total_samples']).isf(kstest_stat_cutoff_2)
main_df['beat_all_priors'] = (main_df['kstest_stat_best'] < np.minimum.reduce([main_df['kstest_stat_gaussian'], main_df['kstest_stat_laplace'], main_df['kstest_stat_t']])).astype(int)


frequency_map = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")).set_index(['dataset', 'transform', 'group'])
# main_df = main_df.set_index(['dataset', 'subset', 'transform', 'group']).merge(frequency_map, left_index = True, right_index=True).reset_index()
main_df = main_df.set_index(['dataset', 'subset', 'transform', 'group']).reset_index() #[(main_df['dataset'] == 'pastis') | (main_df['dataset'] == 'agriVision') | (main_df['dataset'] == 'spaceNet')]
print(main_df.shape)
main_df = main_df.merge(pd.read_csv('result_categorization - combined_categories.csv')[['github_plot', 'failure_category', 'failure_type', 'which_ones']], on='github_plot', how='left')
main_df['dataset'].value_counts()
main_df.head()

/Users/brandonmarks/opt/anaconda3/envs/HBMV/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:269: RuntimeWarning: divide by zero encountered in divide
  return (0.5/(n if not isinstance(n, Iterable) else np.asanyarray(n)),
/Users/brandonmarks/opt/anaconda3/envs/HBMV/lib/python3.10/site-packages/scipy/stats/_continuous_distns.py:269: RuntimeWarning: divide by zero encountered in divide
  return (0.5/(n if not isinstance(n, Iterable) else np.asanyarray(n)),


(733, 38)


,dataset,subset,transform,group,obs_var,var_lower,var_upper,obs_kurt,kurt_lower,kurt_upper,...,dataset_type,hull,best_beta,best_1/beta,kstest_stat_cutoff_0.10,kstest_stat_cutoff_0.20,beat_all_priors,failure_category,failure_type,which_ones
0,syntheticMRI3D,full,wavelet,2,NaN,NaN,NaN,NaN,NaN,NaN,...,medical,NaN,NaN,NaN,NaN,NaN,0,trivial_failure,low samples,NaN
1,syntheticMRI3D,full,wavelet,3,1917588.40,1678674.100,2165048.500,-0.628360,-0.869535,-0.348175,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x7...,21.078431,0.047442,0.160438,0.140370,0,trivial_failure,multimodal,NaN
2,syntheticMRI3D,full,wavelet,4,814282.00,777582.600,851497.500,1.116847,0.878992,1.405564,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x7...,3.333333,0.300000,0.025709,0.022526,0,interesting_failure,double inflection,NaN
3,syntheticMRI3D,full,wavelet,5,249365.95,231914.970,267766.940,5.788045,4.961286,6.675331,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x7...,73.000000,0.013699,0.007099,0.006222,1,practically_pass,pass,NaN
4,syntheticMRI3D,full,wavelet,6,43869.16,40573.156,47332.902,6.932356,5.826878,8.281218,...,medical,<scipy.spatial._qhull.ConvexHull object at 0x7...,10.000000,0.100000,0.002280,0.001998,1,practically_pass,pass,NaN


In [ ]:
#.groupby(["dataset", "subset", "transform", ])
failCatDF = main_df.copy()
failCatDF["ones"] = 1
failCatDF["pass"] = failCatDF["failure_category"].apply(lambda x: x in ["practically_pass", "actually_pass"])
failCatDF = failCatDF[["dataset_type","dataset", "subset", "transform", "orientation", "failure_category", "ones", "pass"]]
failCatDF.fillna("None", inplace=True)
failCatDF = failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"]).count()
failCatDF = failCatDF.reset_index()
# Calculate pass percentage for each group

failCatDF

,dataset_type,dataset,subset,transform,orientation,failure_category,ones,pass,pass_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,practically_pass,6,6,6
1,medical,syntheticMRI2D,axial,wavelet,diagonal,trivial_failure,2,2,2
2,medical,syntheticMRI2D,axial,wavelet,horizontal,practically_pass,5,5,5
3,medical,syntheticMRI2D,axial,wavelet,horizontal,trivial_failure,3,3,3
4,medical,syntheticMRI2D,axial,wavelet,vertical,practically_pass,5,5,5
...,...,...,...,...,...,...,...,...,...
82,remote sensing,spaceNet,full,wavelet,diagonal,TO DISCUSS,12,12,12
83,remote sensing,spaceNet,full,wavelet,diagonal,actually_pass,3,3,3
84,remote sensing,spaceNet,full,wavelet,diagonal,practically_pass,17,17,17
85,remote sensing,spaceNet,full,wavelet,horizVert,TO DISCUSS,12,12,12


In [66]:
failCatDF = main_df.copy()
failCatDF["pass"] = failCatDF["failure_category"].apply(lambda x: x in ["practically_pass", "actually_pass"])
failCatDF = failCatDF[["dataset_type","dataset", "subset", "transform", "orientation", "failure_category", "pass"]]
failCatDF.fillna("None", inplace=True)
failCatDF["number"] = (
    failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])["pass"]
    .transform("count")
)

failCatDF["pass_percentage"] = np.round(
    failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"])["pass"]
    .transform("mean") * 100
)

failCatDF = failCatDF.groupby(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"]).first().reset_index().sort_values(by=["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])
failCatDF


,dataset_type,dataset,subset,transform,orientation,failure_category,pass,number,pass_percentage
0,medical,syntheticMRI2D,axial,wavelet,diagonal,practically_pass,True,6,75.0
1,medical,syntheticMRI2D,axial,wavelet,diagonal,trivial_failure,False,2,75.0
2,medical,syntheticMRI2D,axial,wavelet,horizontal,practically_pass,True,5,62.0
3,medical,syntheticMRI2D,axial,wavelet,horizontal,trivial_failure,False,3,62.0
4,medical,syntheticMRI2D,axial,wavelet,vertical,practically_pass,True,5,62.0
...,...,...,...,...,...,...,...,...,...
82,remote sensing,spaceNet,full,wavelet,diagonal,TO DISCUSS,False,12,62.0
83,remote sensing,spaceNet,full,wavelet,diagonal,actually_pass,True,3,62.0
84,remote sensing,spaceNet,full,wavelet,diagonal,practically_pass,True,17,62.0
85,remote sensing,spaceNet,full,wavelet,horizVert,TO DISCUSS,False,12,62.0


# Remote Sensing

In [67]:
DATASET_TYPE = "remote sensing" 

In [68]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage"]]
medical

pass_percentage
dataset_type   dataset    subset transform orientation                 
remote sensing agriVision full   fourier   None                    95.0
                                 wavelet   diagonal               100.0
                                           horizVert              100.0
               pastis     full   fourier   None                   100.0
                                 wavelet   diagonal               100.0
                                           horizVert              100.0
               spaceNet   full   fourier   None                   100.0
                                 wavelet   diagonal                62.0
                                           horizVert               62.0

In [69]:

medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical.set_index(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])[["number"]]


number
dataset_type   dataset    subset transform orientation failure_category        
remote sensing agriVision full   fourier   None        TO DISCUSS             2
                                                       actually_pass          1
                                                       practically_pass      41
                                 wavelet   diagonal    actually_pass          6
                                                       practically_pass      30
                                           horizVert   actually_pass          4
                                                       practically_pass      32
               pastis     full   fourier   None        actually_pass         40
                                                       practically_pass       4
                                 wavelet   diagonal    actually_pass         16
                                                       practically_pass      12
                                           horizVert   actually_pass         16
                                                       practically_pass      12
               spaceNet   full   fourier   None        practically_pass      32
                                 wavelet   diagonal    TO DISCUSS            12
                                                       actually_pass          3
                                                       practically_pass      17
                                           horizVert   TO DISCUSS            12
                                                       practically_pass      20

# Natural Images

In [70]:
DATASET_TYPE = "natural" 

In [71]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage"]]
medical

pass_percentage
dataset_type dataset         subset  transform orientation                 
natural      coco            indoor  wavelet   diagonal                22.0
                                               horizontal               0.0
                                               vertical                25.0
                             outdoor wavelet   diagonal                25.0
                                               horizontal               0.0
                                               vertical                28.0
             segmentAnything full    wavelet   diagonal                36.0
                                               horizontal               0.0
                                               vertical                42.0

In [72]:

medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical.set_index(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])[["number"]]


number
dataset_type dataset         subset  transform orientation failure_category        
natural      coco            indoor  wavelet   diagonal    TO DISCUSS            13
                                                           actually_pass          7
                                                           trivial_failure       12
                                               horizontal  TO DISCUSS             3
                                                           trivial_failure       29
                                               vertical    TO DISCUSS             3
                                                           actually_pass          8
                                                           trivial_failure       21
                             outdoor wavelet   diagonal    TO DISCUSS             4
                                                           actually_pass          8
                                                           trivial_failure       20
                                               horizontal  TO DISCUSS             9
                                                           trivial_failure       23
                                               vertical    TO DISCUSS             4
                                                           actually_pass          8
                                                           practically_pass       1
                                                           trivial_failure       19
             segmentAnything full    wavelet   diagonal    TO DISCUSS             8
                                                           actually_pass          8
                                                           practically_pass       5
                                                           trivial_failure       15
                                               horizontal  TO DISCUSS            14
                                                           trivial_failure       22
                                               vertical    TO DISCUSS             9
                                                           actually_pass          9
                                                           practically_pass       6
                                                           trivial_failure       12

# Medical

In [73]:
DATASET_TYPE = "medical"

In [74]:
medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical = medical.groupby(["dataset_type", "dataset", "subset", "transform", "orientation"]).first()[["pass_percentage"]]
medical

pass_percentage
dataset_type dataset        subset   transform orientation                 
medical      syntheticMRI2D axial    wavelet   diagonal                75.0
                                               horizontal              62.0
                                               vertical                62.0
                            coronal  wavelet   diagonal                88.0
                                               horizontal              62.0
                                               vertical                62.0
                            sagittal wavelet   diagonal                62.0
                                               horizontal              62.0
                                               vertical                88.0
             syntheticMRI3D full     wavelet   aad                     57.0
                                               ada                     57.0
                                               add                     57.0
                                               daa                     43.0
                                               dad                     57.0
                                               dda                     71.0
                                               ddd                     71.0

In [75]:

medical = failCatDF.copy()
medical = medical[medical["dataset_type"] == DATASET_TYPE]
medical.set_index(["dataset_type", "dataset", "subset", "transform", "orientation", "failure_category"])[["number"]]


number
dataset_type dataset        subset   transform orientation failure_category           
medical      syntheticMRI2D axial    wavelet   diagonal    practically_pass          6
                                                           trivial_failure           2
                                               horizontal  practically_pass          5
                                                           trivial_failure           3
                                               vertical    practically_pass          5
                                                           trivial_failure           3
                            coronal  wavelet   diagonal    actually_pass             2
                                                           practically_pass          5
                                                           trivial_failure           1
                                               horizontal  interesting_failure       1
                                                           practically_pass          5
                                                           trivial_failure           2
                                               vertical    practically_pass          5
                                                           trivial_failure           3
                            sagittal wavelet   diagonal    practically_pass          5
                                                           trivial_failure           3
                                               horizontal  interesting_failure       1
                                                           practically_pass          5
                                                           trivial_failure           2
                                               vertical    practically_pass          7
                                                           trivial_failure           1
             syntheticMRI3D full     wavelet   aad         interesting_failure       1
                                                           practically_pass          4
                                                           trivial_failure           2
                                               ada         interesting_failure       1
                                                           practically_pass          4
                                                           trivial_failure           2
                                               add         practically_pass          4
                                                           trivial_failure           3
                                               daa         interesting_failure       1
                                                           practically_pass          3
                                                           trivial_failure           3
                                               dad         actually_pass             1
                                                           interesting_failure       1
                                                           practically_pass          3
                                                           trivial_failure           2
                                               dda         practically_pass          5
                                                           trivial_failure           2
                                               ddd         actually_pass             2
                                                           practically_pass          3
                                                           trivial_failure           2